#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime

import os
import pandas as pd
import xmlrpc.client


# Добавление в лог через сервер
def add_log(event, event_time, work_time):
    try:
        stat_server.add_log(event, event_time, work_time)
        return True
    except ConnectionRefusedError:
        return False

def is_server_working():
    try:
        server.system.listMethods()
        return True
    except ConnectionRefusedError:
        return False

server = xmlrpc.client.ServerProxy("http://localhost:8008")
stat_server = xmlrpc.client.ServerProxy("http://localhost:8038")

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

class ProxyServer(SimpleXMLRPCServer):

    def _dispatch(self, method, params):
        if not is_server_working():
            return "server is not working"
        start_time = datetime.datetime.now()
        func = getattr(server, method)
        result = func(*params)
        work_time = (datetime.datetime.now()-start_time).total_seconds()
        event_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        add_log(method, event_time, work_time)
        return result
        # return execute(method, *params)

proxy_server = ProxyServer(("localhost", 8028),
                            requestHandler=RequestHandler)
proxy_server.register_introspection_functions()

print ("Listening on port 8028...")
proxy_server.serve_forever()


# def execute(fun_name, *args):
#     methods = getListMethods()
#     if methods is None:
#         raise Exception("server is not working")
#     if not fun_name in methods:
#         raise AttributeError("unsupported method")
#     stringify = lambda x: '"'+x+'"' if type(x) is str else str(x)
#     list_str_args = map(stringify,args)
#     string_from_args = ', '.join(list_str_args)
#     return eval('server.'+fun_name+'('+string_from_args+')')

# def execute(fun_name, *args):
#     methods = getListMethods()
#     if methods is None:
#         raise ValueError("server is not working")
#     if not fun_name in methods:
#         raise AttributeError("unsupported method")
#     return eval('server.'+fun_name)(*args)


Listening on port 8028...


127.0.0.1 - - [11/Oct/2023 12:03:17] "POST /RPC2 HTTP/1.1" 200 -
